# Applied Data Science Capstone
## Comparison of Cities - Karachi VS Islamabad
### Part of IBM Data Science Professional Specialization on Coursera

### Introduction:

During this capstone project we would be looking at Real Estate industry's data of two major cities in Pakistan, Karachi and Islamabad.

Karachi is the industrial hub of Pakistan with over 14.9 Million people of various groups, affiliations, backgrounds and ethnicities residing there.

On the other hand Islamabad is the capital of Pakistan which has seen growth in Real Estate industry in past 1-2 years as compared to other cities.

#### Data & Source:

Our dataset is taken from an online Real Estate portal "Zameen.com" which lists various real estate listing in various cities, including Karachi and Islamabad. The dataset has location data (longitude & latitude) for each listing, the type, size, value and amenities of each listing.

We shall utilize Foursquare API for segmentation and then use clustering algorithms to determine commonalities between various localities in our listing dataset.

#### Stakeholders:

This report will help potential buyers and investors in making educated decisions regarding their purchases or investment. Stakeholders for this report will be buyers, investors and Real Estate marketing portals.